# Quantile Trading

## Utility Functions

In [ ]:
# <include-accumulation_opportunity/utils.py>

## Imports

In [ ]:
# <imports>
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import trange

from accumulation_opportunity import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "none"

## Summary

In [ ]:
df = utils.get_trade_data("BTC-USD", "2018")
df.name = "BTC-USD"

In [ ]:
df_c = df.loc["2018-04-08 22:05":"2018-04-08 22:10"]
fig = px.scatter(df_c, x=df_c.index, y=df_c.PriceMillionths, color=df_c.Side.astype(str))
fig.update_traces(marker=dict(size=3)),
fig.show()

In [ ]:
fig = px.scatter(df_c, x=df_c.index, y=(df_c.SizeBillionths * df_c.Side), color=df_c.Side.astype(str))
fig.show()

In [ ]:
df

In [ ]:
df_15min = df.groupby("Side").resample("15min").sum()[["SizeBillionths"]]
df_15min = df_15min.reset_index()
df_15min.SizeBillionths = df_15min.SizeBillionths * df_15min.Side
df_15min = df_15min.sort_values(["Side", "timestamp_utc_nanoseconds"], ascending=[False, True])
df_15min.name = df.name
df_15min.plot(x="timestamp_utc_nanoseconds", y="SizeBillionths", kind="bar", title=f"{df_15min.name}: Total Volume Traded in 15 Minute Intervals", color=df_15min.Side.astype(str), labels={"color": "Side"})


In [ ]:
df_15min.groupby(["Side", "timestamp_utc_nanoseconds"]).sum().unstack("Side").describe()

\[OUT OF DATE]

Here we think about setting the chunk size such that some average number of trades will take place over a 15 minute window.
* Starting on the buy side (Side=1), if we want there to be 20 trades over 15 minutes, with an average of volume of 6.5e10, we would set the chunk size at 3.25e9
* To set the quantity, $Q$, to be of consequence, we consider ranges up to our total participation rate

In [ ]:
arrival_time = "2018-04-08 22:05"
participation = .050
max_trade_participation = 0.10

chunk_size = 6.5e9
quantity = 3.25e9
side = 1
price_window_ms = 200

In [ ]:
def get_accum_df(
    df: pd.DataFrame,
    side: int = 1,
    arrival_time: str = "2018-04-08 22:05",
    participation = .050,
    max_trade_participation = 0.10,
    chunk_size = 6.5e9,
    quantity = 3.25e9,
    price_window_ms = 200
):
    df_accum = df.loc[arrival_time:].copy()
    df_accum["CumSizeBillionths"] = df_accum.SizeBillionths.cumsum()

    df_accum = df_accum[df_accum.Side == side].copy()

    price_agg = "max" if side else "min"
    df_accum = (
        df_accum.reset_index().groupby("timestamp_utc_nanoseconds")
        .agg({"CumSizeBillionths": "max", "SizeBillionths": "sum", "PriceMillionths": price_agg })
        )

    df_accum["CumChunks"] = df_accum.CumSizeBillionths.floordiv(chunk_size)
    df_accum["CumParticipation"] = (
        df_accum.CumChunks.map(
            df_accum.groupby("CumChunks").min()["CumSizeBillionths"].iloc[1:])
            * participation).fillna(0).apply(lambda x: min(x, quantity)
            )

    df_accum["TradePrice"] = df_accum.PriceMillionths.sort_index(ascending=False).rolling(f"{price_window_ms}ms").agg(price_agg).sort_index()

    df_accum["QualifiedTrade"] = df_accum["TradePrice"] == df_accum["PriceMillionths"]

    df_accum["MaxTradeSize"] = df_accum.SizeBillionths * max_trade_participation

    return df_accum

In [ ]:
df_accum_buy, df_accum_sell = [get_accum_df(df, s) for s in [1, -1]]

In [ ]:
def get_trades(df_accum: pd.DataFrame):
    trades = []
    cum_trades = 0
    tick_idx = 0
    while cum_trades < df_accum.CumParticipation.max():
        tick = df_accum.iloc[tick_idx]
        
        if tick.CumParticipation - cum_trades > 0 and tick.QualifiedTrade:
            trade_size = min(tick.CumParticipation - cum_trades, tick.MaxTradeSize)
            trades.append((tick.name, trade_size, tick.TradePrice))
            cum_trades += trade_size
        
        tick_idx += 1
    return trades

In [ ]:
df_trades = pd.DataFrame(get_trades(df_accum_buy), columns=["timestamp_utc_nanoseconds", "TradeSize", "TradePrice"]).set_index("timestamp_utc_nanoseconds").astype(int)

In [ ]:
df_trades.TradeSize.sum()

In [ ]:
columns = ["PriceMillionths", "SizeBillionths", "Side"]
df_result = df[:df_trades.index.max()].copy()

fig = go.Figure()
fig.add_scatter(x=df_result.index, y=df_result.PriceMillionths, mode="markers", name="Buy", marker=dict(size=7, color=utils.COLORS[2]))
fig.add_scatter(x=df[arrival_time:df_trades.index.max()].index, y=df_result[df_result.Side == -1].PriceMillionths, mode="markers", name="Sell", marker=dict(size=7, color=utils.COLORS[4]))
fig.add_scatter(x=df_trades.index, y=df_trades.TradePrice, mode="markers", name="Trade - Buy", marker=dict(size=4, color=utils.COLORS[1]))
fig.update_layout(
    title="Trade Prices",
    xaxis_title="timestamp_utc_nanoseconds",
    yaxis_title="PriceMillionths"
    )
fig.show()

In [ ]:
bar_width = 2000

fig = go.Figure()
fig.add_bar(x=df_result[df_result.Side == 1].index, y=df_result[df_result.Side == 1].SizeBillionths,name="Buy", marker_color=utils.COLORS[0], width=bar_width)
fig.add_bar(x=df_result[df_result.Side == -1].index, y=df_result[df_result.Side == -1].SizeBillionths * -1, name="Sell", marker_color=utils.COLORS[4], width=bar_width)
fig.add_bar(x=df_trades.index, y=df_trades.TradeSize,name="Trade - Buy", marker_color=utils.COLORS[1], width=bar_width)
fig.update_layout(
    title="Trade Sizes",
    xaxis_title="timestamp_utc_nanoseconds",
    yaxis_title="SizeBillionths"
    )
# fig.update_traces(marker=dict(size=3)),
fig.show()

In [ ]:
fig = go.Figure()
# fig.add_scatter(x=df_result.index, y=df_result.CumSizeBillionths, name="Buy + Sell", line=dict(color=utils.COLORS[2]))
fig.add_scatter(x=df_result.index, y=df_result.CumParticipation, name="Target Participation", line=dict(color=utils.COLORS[3]))
fig.add_scatter(x=df_trades.index, y=df_trades.TradeSize.cumsum(), name="Cumulative Trades", line=dict(color=utils.COLORS[1]))

# fig.add_scatter(x=df_c.index, y=df_c.SizeBillionths.cumsum(), name="Buy", line=dict(color=utils.COLORS[0]))
fig.update_layout(title="Target Participation vs. Cumulative Trades")